In [ ]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import pandas as pd 
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import TensorDataset
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os
import copy

from PIL import Image
import imageio

import statsmodels.api as sm
import statsmodels.formula.api as smf